# Lab 07

## Zadanie 1

Przygotuj demo programu znajdującego miejsca zerowe metodą Newtona. Wykorzystując `argparse` ([link](https://docs.python.org/3.6/library/argparse.html#module-argparse)) lub `optparse` ([link](https://docs.python.org/3.6/library/optparse.html)) obsłuż:
- ustalane punktu startowego,
- wielkość kroku w pochodnej,
- ilość kroków metody,
- dokładność
- pomoc

Program uruchamiamy podając, np.:

`./newton.py x**2+x+1 -h 0.00001`

## Zadanie 2

Zadanie należy wykonać wykorzystując program `BagOfWords` z poprzednich zajęć.


Przerób ją tak, żeby interpunkcja, cyfry i wszelkie inne znaki nie przeszkadzały w parsowaniu tekstu. Uruchom ja na [tekście hamleta](http://www.gutenberg.org/cache/epub/1787/pg1787.txt). Ile razy występuje słowo hamlet? Jak brzmi 10 najczęściej występujących słów?

In [2]:
import requests

response = requests.get('https://www.gutenberg.org/cache/epub/1787/pg1787.txt')
if response.status_code == 200:
    with open('hamlet.txt', 'wb+') as f:
        f.write(response.content)

## Zadanie 3

Wykorzystując `pickle` zapisz i odczytaj klasy z poprzedniego zadania nakarmonej Hamletem. Porównaj metody i rozmiar.

In [90]:
from collections import defaultdict
from io import TextIOWrapper

class BagOfWords:    
    def __init__(self, document = None):
        self.word_count = defaultdict(self._default_value)
        if isinstance(document, str):
            for word in document.split(' '):
                    self.word_count[word] += 1
        elif isinstance(document, TextIOWrapper):
            for line in document:
                for word in line.rstrip().split(' '):
                    self.word_count[word] += 1

    def __repr__(self):
        str_repr = ''
        for key, value in sorted(self.word_count.items(), key=lambda x: (-x[1], x[0])):
            str_repr += f'{key}:{value}, '
        return str_repr[0:len(str_repr)-2]

    def __iter__(self):
        for item in sorted(self.word_count.items(), key=lambda x: (-x[1], x[0])):
            yield item[0]
    
    def __contains__(self, item):
        return item in self.word_count
    
    def __add__(self, other_bag):
        new_dict = defaultdict(self._default_value)        
        for key, item in other_bag.word_count.items():
            new_dict[key] += item
        
        for key, item in self.word_count.items():
            new_dict[key] += item
        new_bag = BagOfWords()
        new_bag.word_count = new_dict
        return new_bag

    def __getitem__(self, arg):
        return self.word_count[arg]
    
    def __setitem__(self, key, value):
        self.word_count[key] = value

    def _default_value(self):
            return 0

## Zadanie 4

Wykorzystując [https://gist.github.com/pamelafox/986163](https://gist.github.com/pamelafox/986163) podaj aktualną godzinę we:
- wszystkich krajach, wyświetlając je zgrupowane względem kontynentów,
- przeprowadź symulacyjne wyświetlanie kolejnych krajów w miarę jak w danym kraju wybija północ, opóżnienie wyświetlania ustaw proporcjonalne do realnego czasu

In [88]:
from countries import countries
from itertools import groupby 
import requests
from zoneinfo import ZoneInfo, available_timezones
from datetime import datetime, timezone
response = requests.get('https://gist.githubusercontent.com/pamelafox/986163/raw/eeeabff229831334f8a84bdddd5768cc84c50bd4/countryinfo.py')
if response.status_code == 200:
    with open('countries.py', 'wb+') as f:
        f.write(response.content)


offsets = defaultdict(lambda: set())

for continent, countries in groupby(sorted(countries, key=lambda d: d['continent']), lambda country: country['continent']):
    print(f"{continent}:")
    for country in list(countries):
        print(country['name'], end=', ')
        for zone in country["timezones"]:
            delta = ZoneInfo(zone).utcoffset(datetime.now(ZoneInfo('utc')))
            offset = delta.days * 24 + delta.seconds/3600
            offsets[offset].add(country['name'])
        print()
    print("_____________________________________")


Africa:
Angola, 
Burkina Faso, 
Burundi, 
Benin, 
Botswana, 
Democratic Republic of the Congo, 
Republic of the Congo, 
CÃ´te d'Ivoire, 
Cameroon, 
Cape Verde, 
Djibouti, 
Egypt, 
Eritrea, 
Ethiopia, 
Gabon, 
Ghana, 
The Gambia, 
Guinea, 
Guinea-Bissau, 
Kenya, 
Liberia, 
Lesotho, 
Libya, 
Madagascar, 
Mali, 
Mauritania, 
Mauritius, 
Malawi, 
Mozambique, 
Namibia, 
Niger, 
Nigeria, 
Rwanda, 
Seychelles, 
Sudan, 
Sierra Leone, 
Senegal, 
Somalia, 
SÃ£o TomÃ© and PrÃ­ncipe, 
Togo, 
Tunisia, 
Tanzania, 
Uganda, 
Zambia, 
Zimbabwe, 
Algeria, 
Central African Republic, 
Chad, 
Comoros, 
Equatorial Guinea, 
Morocco, 
South Africa, 
Swaziland, 
_____________________________________
Asia:
Afghanistan, 
Armenia, 
Azerbaijan, 
Bangladesh, 
Bahrain, 
Brunei Darussalam, 
Bhutan, 
People's Republic of China, 
Cyprus, 
Georgia, 
Indonesia, 
Israel, 
India, 
Iraq, 
Iran, 
Jordan, 
Japan, 
Kyrgyzstan, 
North Korea, 
South Korea, 
Kuwait, 
Lebanon, 
Myanmar, 
Mongolia, 
Maldives, 
Malaysia, 
Nepal, 
Om

In [89]:
from time import sleep
sorted_timezone_offsets = sorted(offsets.items(),key=lambda item: item[0], reverse=True)
for i, (key, country) in enumerate(sorted_timezone_offsets):
    print(country)
    if i + 1 < len(sorted_timezone_offsets):
        sleep(key - sorted_timezone_offsets[i+1][0])

{'Kiribati'}
{'New Zealand'}
{'New Zealand', 'Tonga', 'Samoa', 'Kiribati'}
{'Russia', 'Tuvalu', 'Marshall Islands', 'Fiji', 'Kiribati', 'Nauru'}
{'Russia', 'Australia', 'Vanuatu', 'Solomon Islands', 'Federated States of Micronesia'}
{'Australia'}
{'Papua New Guinea', 'Australia', 'Federated States of Micronesia', 'Russia'}
{'Australia'}
{'South Korea', 'Russia', 'Palau', 'East Timor', 'North Korea', 'Indonesia', 'Japan'}
{'Brunei Darussalam', 'Mongolia', 'Russia', "People's Republic of China", 'Philippines', 'Australia', 'Malaysia', 'Indonesia', 'Singapore'}
{'Mongolia', 'Russia', 'Vietnam', 'Indonesia', 'Laos', 'Thailand', 'Cambodia'}
{'Myanmar'}
{'Bhutan', 'Kyrgyzstan', 'Kazakhstan', 'Russia', 'Bangladesh', "People's Republic of China"}
{'Nepal'}
{'Sri Lanka', 'India'}
{'Turkmenistan', 'Kazakhstan', 'Russia', 'Maldives', 'Uzbekistan', 'Pakistan', 'Tajikistan'}
{'Afghanistan'}
{'United Arab Emirates', 'Oman', 'Russia', 'Azerbaijan', 'Mauritius', 'Seychelles', 'Armenia', 'Georgia'}
{'I

## Zadanie 5

Dla klasy `BagOfWords` napisz metody `save` oraz `load` wykorzystujące `json`'a do zapisu i odczytu danych.

In [94]:
import json 

class JSONBagOfWords(BagOfWords):

    def __init__(self, *args):
        super().__init__( *args)

    def load(self, json_file):
        with open(json_file) as f:
            for word, count in json.load(f).items(): 
                self.word_count[word] += count

    def save(self, json_file):
        with open(json_file, 'w') as f:
            json.dump(self.word_count, f)

example = JSONBagOfWords()
example.load("words.json") 
print(example)
example.save("output.json")

ala:3, ma:3, kota:2, tez:1, tomek:1
